In [39]:
using JuMP, Cbc, CSV, DataFrames, Plots

data = CSV.read("data2.csv", DataFrame, delim = ',') #Importing dataset

playerMatrix = convert(Array, data[1:end, 1]) #creating matrix of players for parameters

players = convert(Array,data[1:end,1]) 
teams = convert(Array,data[1:end,2]) # teams array
points = convert(Array,data[1:end,3]) # points array
cost = convert(Array,data[1:end,4]) # cost array
position = convert(Array,data[1:end,5]) # position array
name = convert(Array,data[1:end,6]) #name array to convert player1, player2 etc back to names

allteams = Set(teams) #unique set of teams

positionParam = Dict(zip(playerMatrix[1:end], position)) #position parameter
pointsParam = Dict(zip(playerMatrix[1:end], points)) #points parameter
costParam = Dict(zip(playerMatrix[1:end], cost)) # cost parameter
teamParam = Dict(zip(playerMatrix[1:end], teams)) # team parameter
nameParam = Dict(zip(playerMatrix[1:end], name)) # name parameter

gkRequired = 1 #constants for number of goalkeepers
defRequired = 4 #constants for number of defenders
midRequired = 4 #constants for number of midfielders
fwdRequired = 2 #constants for number of forwards

m = Model()

@variable(m, x[players], Bin)

@objective(m, Max, sum(x[i]*pointsParam[i] for i in players))

@constraint(m, budget, sum(x[i]*costParam[i] for i in players) <= 82)
@constraint(m, maxplayers[i in players], sum(x[players])  == 11)
@constraint(m, sameteamscons[j in allteams], sum(x[i] for i in players if teamParam[i] == j) <= 3)
@constraint(m, goalkeepers, sum(x[i] for i in players if positionParam[i] == "GK") == gkRequired)
@constraint(m, defenders, sum(x[i] for i in players if positionParam[i]  == "DEF") == defRequired)
@constraint(m, midfielders, sum(x[i] for i in players if positionParam[i] == "MID") == midRequired)
@constraint(m, forwards, sum(x[i] for i in players if positionParam[i] == "FWD") == fwdRequired)

    
set_optimizer(m, Cbc.Optimizer)
set_silent(m)
optimize!(m)

println("Total Points for Fantasy Team: ", objective_value(m))
println("")

totalCost = 0
for i in players
    if value(x[i]) > 0.99
        println("Player Name: ", nameParam[i])
        println("Team Name: ", teamParam[i])
        println("Points: ", pointsParam[i])
        println("Position: ", positionParam[i])
        println("Cost: ", costParam[i])
        totalCost += costParam[i]
        println("------------")
    end
end

println("Total Cost of Fantasy Team: ", totalCost)

Total Points for Fantasy Team: 1022.0

Player Name: Sá
Team Name: Wolves
Points: 77
Position: GK
Cost: 6.1
------------
Player Name: Alexander-Arnold
Team Name: Liverpool
Points: 108
Position: DEF
Cost: 6.8
------------
Player Name: Cancelo
Team Name: Man City
Points: 91
Position: DEF
Cost: 6.82
------------
Player Name: van Dijk
Team Name: Liverpool
Points: 83
Position: DEF
Cost: 6.83
------------
Player Name: Rüdiger
Team Name: Chelsea
Points: 81
Position: DEF
Cost: 6.85
------------
Player Name: Salah
Team Name: Liverpool
Points: 160
Position: MID
Cost: 6.302
------------
Player Name: Son
Team Name: Spurs
Points: 81
Position: MID
Cost: 6.307
------------
Player Name: Bernardo
Team Name: Man City
Points: 91
Position: MID
Cost: 6.315
------------
Player Name: Gallagher
Team Name: Crystal Palace
Points: 89
Position: MID
Cost: 6.342
------------
Player Name: Vardy
Team Name: Leicester
Points: 77
Position: FWD
Cost: 6.573
------------
Player Name: Dennis
Team Name: Watford
Points: 84
Pos